# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.12,-134.97,70.72,71,0,21.99,PF,1597020579
1,1,Albany,42.60,-73.97,81.00,63,0,4.45,US,1597020579
2,2,Punta Arenas,-53.15,-70.92,33.80,80,100,16.11,CL,1597020579
3,3,Deputatsky,69.30,139.90,37.69,95,100,6.33,RU,1597020579
4,4,Airai,-8.93,125.41,74.05,56,3,0.83,TL,1597020579
...,...,...,...,...,...,...,...,...,...,...
564,564,Mandalgovi,45.76,106.27,59.11,57,96,14.79,MN,1597020759
565,565,Mingoyo,-10.10,39.63,66.20,93,20,6.93,TZ,1597020759
566,566,Mugur-Aksy,50.35,90.50,54.82,63,100,1.30,RU,1597020759
567,567,Itaituba,-4.28,-55.98,77.14,49,0,0.20,BR,1597020759


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Store 'Lat' and 'Lng' into  locations, and humidity column into humidity 
locations = city_data_df[["Lat", "Lng"]].astype(float)
humidity = city_data_df["Humidity"]

# Create a humidity Heatmap
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
